In [31]:
import copy
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt

from alns import ALNS
from alns.accept import RecordToRecordTravel
from alns.select import RouletteWheel
from alns.stop import MaxIterations

## Simple data

In [32]:
# distance_matrix = [
#     [0, 50, 80, 80, 50, 80, 80, 82, 84, 86],        # depot 0
#     [50, 0, 0, 80, 80, 80, 50, 50, 90, 90],         # station 1
#     [80, 70, 0, 50, 80, 80, 80, 80, 80, 80],        # station 2
#     [80, 80, 50, 0, 50, 80, 50, 50, 80, 80],        # station 3
#     [50, 80, 80, 50, 0, 70, 90, 80, 30, 30],        # station 4
#     [80, 80, 80, 80, 70, 0, 80, 80, 80, 80],        # station 5
#     [80, 50, 80, 50, 90, 80, 0, 80, 90, 90],        # customer 6
#     [82, 50, 80, 50, 80, 80, 80, 0, 90, 90],        # customer 7
#     [84, 90, 80, 80, 30, 80, 90, 90, 0, 30],        # customer 8
#     [86, 90, 80, 80, 30, 80, 90, 90, 30, 0]         # customer 9
# ]
#
# nodes = list(range(len(distance_matrix)))  # [0, 1, ..., 9]
# depot = 0
# charging_stations = [1, 2, 3, 4, 5]
# customers_robot_only = [6, 8, 9]    # chỉ robot giao
# customers_both = [7]                # van hoặc robot đều giao
# all_customers = customers_robot_only + customers_both
#
# # Tham số của van và robot
# van_params = {
#     "speed": 2.0,
#     "battery_capacity": 400.0,
#     "capacity": 200.0,
#     "travel_cost_rate": 2.0,
#     "energy_consumption_rate": 2.0,
#     "charge_rate": 10.0
# }
#
# robot_params = {
#     "speed": 1.0,
#     "battery_capacity": 120.0,
#     "capacity": 50.0,
#     "travel_cost_rate": 1.0,
#     "energy_consumption_rate": 1.0,
#     "charge_rate": 4.0
# }

## Define problem

In [33]:
from alns import ALNS, State
from alns.accept import SimulatedAnnealing
from alns.select import RouletteWheel
from alns.stop import MaxIterations
import numpy as np
import random
import copy
import math

# Data from user
distance_matrix = np.array([
    [0, 50, 80, 80, 50, 80, 80, 82, 84, 86],        # depot 0
    [50, 0, 0, 80, 80, 80, 50, 50, 90, 90],         # station 1
    [80, 70, 0, 50, 80, 80, 80, 80, 80, 80],        # station 2
    [80, 80, 50, 0, 50, 80, 50, 50, 80, 80],        # station 3
    [50, 80, 80, 50, 0, 70, 90, 80, 30, 30],        # station 4
    [80, 80, 80, 80, 70, 0, 80, 80, 80, 80],        # station 5
    [80, 50, 80, 50, 90, 80, 0, 80, 90, 90],        # customer 6
    [82, 50, 80, 50, 80, 80, 80, 0, 90, 90],        # customer 7
    [84, 90, 80, 80, 30, 80, 90, 90, 0, 30],        # customer 8
    [86, 90, 80, 80, 30, 80, 90, 90, 30, 0]         # customer 9
])

nodes = list(range(len(distance_matrix)))
depot = 0
charging_stations = [1, 2, 3, 4, 5]
customers_robot_only = [6, 8, 9]
customers_both = [7]
all_customers = customers_robot_only + customers_both

van_params = {
    "speed": 2.0,
    "battery_capacity": 400.0,
    "capacity": 200.0,
    "travel_cost_rate": 2.0,
    "energy_consumption_rate": 2.0,
    "charge_rate": 10.0
}

robot_params = {
    "speed": 1.0,
    "battery_capacity": 120.0,
    "capacity": 30.0,
    "travel_cost_rate": 1.0,
    "energy_consumption_rate": 1.0,
    "charge_rate": 4.0
}

customer_demand = {
    6: 15,
    7: 20,
    8: 10,
    9: 25
}

class Route:
    """
    Represents a route for a vehicle (van or robot)
    """
    def __init__(self, route_type, nodes=None):
        """
        Initialize a route

        Args:
            route_type: 'van', 'robot', or 'van-robot'
            nodes: List of nodes in the route
        """
        self.route_type = route_type
        self.nodes = nodes if nodes else []
        self.load = 0
        self.energy_consumption = 0
        self.distance = 0
        self.travel_time = 0

    def copy(self):
        """Create a deep copy of the route"""
        new_route = Route(self.route_type, self.nodes.copy())
        new_route.load = self.load
        new_route.energy_consumption = self.energy_consumption
        new_route.distance = self.distance
        new_route.travel_time = self.travel_time
        return new_route

    def update_metrics(self):
        """Update route metrics: distance, energy consumption, travel time"""
        self.distance = 0
        if len(self.nodes) <= 1:
            return

        # Calculate route distance
        for i in range(len(self.nodes) - 1):
            self.distance += distance_matrix[self.nodes[i]][self.nodes[i+1]]

        # Calculate energy consumption based on route type
        if self.route_type == 'van':
            self.energy_consumption = self.distance * van_params["energy_consumption_rate"]
            self.travel_time = self.distance / van_params["speed"]
        elif self.route_type == 'robot':
            self.energy_consumption = self.distance * robot_params["energy_consumption_rate"]
            self.travel_time = self.distance / robot_params["speed"]
        elif self.route_type == 'van-robot':
            self.energy_consumption = self.distance * van_params["energy_consumption_rate"]
            self.travel_time = self.distance / van_params["speed"]

class TwoEchelonState(State):
    """
    Represents a solution to the Two-Echelon Electric Vehicle Routing Problem
    """
    def __init__(self):
        """Initialize an empty solution"""
        self.van_routes = []  # List of van routes
        self.robot_routes = []  # List of robot routes
        self.unassigned = all_customers.copy()  # Customers not assigned to any route
        self.total_cost = float('inf')  # Total cost of the solution

    def objective(self):
        """
        Calculate the objective function value (total cost)

        Returns:
            float: Total cost of the solution
        """
        if not self.is_feasible():
            return float('inf')

        # Calculate total distance and energy consumption
        total_distance = 0
        total_energy = 0

        for route in self.van_routes:
            total_distance += route.distance
            total_energy += route.energy_consumption

        for route in self.robot_routes:
            total_distance += route.distance
            total_energy += route.energy_consumption

        # Calculate total cost
        van_cost = total_distance * van_params["travel_cost_rate"]
        robot_cost = sum(r.distance * robot_params["travel_cost_rate"] for r in self.robot_routes)

        self.total_cost = van_cost + robot_cost

        return self.total_cost

    def is_feasible(self):
        """
        Check if the solution is feasible

        Returns:
            bool: True if the solution is feasible, False otherwise
        """
        # Check if all customers are served
        assigned_customers = []
        for route in self.van_routes:
            for node in route.nodes:
                if node in all_customers:
                    assigned_customers.append(node)

        for route in self.robot_routes:
            for node in route.nodes:
                if node in all_customers:
                    assigned_customers.append(node)

        if set(assigned_customers) != set(all_customers):
            return False

        # Check van capacity and battery constraints
        for route in self.van_routes:
            if route.load > van_params["capacity"]:
                return False
            if route.energy_consumption > van_params["battery_capacity"]:
                return False

        # Check robot capacity and battery constraints
        for route in self.robot_routes:
            if route.load > robot_params["capacity"]:
                return False
            if route.energy_consumption > robot_params["battery_capacity"]:
                return False

        return True

    def copy(self):
        """
        Create a deep copy of the state

        Returns:
            TwoEchelonState: A deep copy of the current state
        """
        new_state = TwoEchelonState()
        new_state.van_routes = [route.copy() for route in self.van_routes]
        new_state.robot_routes = [route.copy() for route in self.robot_routes]
        new_state.unassigned = self.unassigned.copy()
        new_state.total_cost = self.total_cost
        return new_state

    def add_customer_to_route(self, customer, route, position):
        """
        Add a customer to a route at a specified position

        Args:
            customer: Customer ID to add
            route: Route object to add the customer to
            position: Position in the route to insert the customer
        """
        route.nodes.insert(position, customer)

        # Update load
        if customer in all_customers:
            route.load += customer_demand[customer]

        # Update other route metrics
        route.update_metrics()

        # Remove from unassigned if in the list
        if customer in self.unassigned:
            self.unassigned.remove(customer)

    def remove_customer_from_route(self, customer, route):
        """
        Remove a customer from a route

        Args:
            customer: Customer ID to remove
            route: Route object to remove the customer from

        Returns:
            bool: True if customer was removed, False otherwise
        """
        if customer not in route.nodes:
            return False

        position = route.nodes.index(customer)
        route.nodes.pop(position)

        # Update load
        if customer in all_customers:
            route.load -= customer_demand[customer]

        # Update other route metrics
        route.update_metrics()

        # Add to unassigned
        if customer in all_customers and customer not in self.unassigned:
            self.unassigned.append(customer)

        return True

    def create_initial_solution(self):
        """
        Create an initial solution using a simple construction heuristic
        """
        # Create one van route from depot to all stations and back
        van_route = Route('van', [depot])

        # Add all stations to the van route
        for station in charging_stations:
            van_route.nodes.append(station)

        # Return to depot
        van_route.nodes.append(depot)
        van_route.update_metrics()
        self.van_routes.append(van_route)

        # Create robot routes from stations to customers
        for station in charging_stations:
            if not self.unassigned:
                break

            robot_route = Route('robot', [station])

            # Sort customers by distance from the station
            sorted_customers = sorted(self.unassigned,
                                     key=lambda c: distance_matrix[station][c])

            for customer in sorted_customers[:2]:  # Assign at most 2 customers per robot
                if customer in self.unassigned and robot_route.load + customer_demand[customer] <= robot_params["capacity"]:
                    self.add_customer_to_route(customer, robot_route, len(robot_route.nodes))

            # Return to a station
            closest_station = min(charging_stations,
                                 key=lambda s: distance_matrix[robot_route.nodes[-1]][s])
            robot_route.nodes.append(closest_station)
            robot_route.update_metrics()

            # Only add non-empty routes (those with customers)
            customers_in_route = [n for n in robot_route.nodes if n in all_customers]
            if customers_in_route:
                self.robot_routes.append(robot_route)

        return self


# ALNS operators
def random_remove(state, random_state):
    """
    Randomly remove customers from the solution

    Args:
        state: Current solution state
        random_state: Random number generator

    Returns:
        TwoEchelonState: Modified solution with customers removed
    """
    destroyed_state = state.copy()

    # Determine number of customers to remove (between 1 and min(30% of total, 3))
    num_remove = random_state.integers(1, min(3, max(1, int(0.3 * len(all_customers)))) + 1)

    # Select random customers to remove
    customers_to_remove = random_state.choice(all_customers, size=num_remove, replace=False)

    # Remove selected customers from routes
    for customer in customers_to_remove:
        # Check van routes
        for route in destroyed_state.van_routes:
            if destroyed_state.remove_customer_from_route(customer, route):
                break

        # Check robot routes
        for route in destroyed_state.robot_routes:
            if destroyed_state.remove_customer_from_route(customer, route):
                break

    return destroyed_state

def worst_remove(state, random_state):
    """
    Remove customers with the highest cost contribution

    Args:
        state: Current solution state
        random_state: Random number generator

    Returns:
        TwoEchelonState: Modified solution with customers removed
    """
    destroyed_state = state.copy()

    # Determine number of customers to remove (between 1 and min(30% of total, 3))
    num_remove = random_state.integers(1, min(3, max(1, int(0.3 * len(all_customers)))) + 1)

    # Calculate cost contribution of each customer
    customer_costs = {}

    for customer in all_customers:
        # Find which route contains this customer
        customer_route = None
        is_van_route = False

        for route in destroyed_state.van_routes:
            if customer in route.nodes:
                customer_route = route
                is_van_route = True
                break

        if not customer_route:
            for route in destroyed_state.robot_routes:
                if customer in route.nodes:
                    customer_route = route
                    break

        if customer_route:
            # Calculate cost with customer
            original_cost = customer_route.distance

            # Remove customer temporarily
            idx = customer_route.nodes.index(customer)
            customer_route.nodes.pop(idx)
            customer_route.update_metrics()

            # Calculate cost without customer
            new_cost = customer_route.distance

            # Reinsert customer
            customer_route.nodes.insert(idx, customer)
            customer_route.update_metrics()

            # Cost contribution
            customer_costs[customer] = original_cost - new_cost

    # Sort customers by cost contribution (descending)
    sorted_customers = sorted(customer_costs.keys(), key=lambda c: -customer_costs[c])

    # Remove worst customers
    customers_to_remove = sorted_customers[:num_remove]

    for customer in customers_to_remove:
        # Check van routes
        for route in destroyed_state.van_routes:
            if destroyed_state.remove_customer_from_route(customer, route):
                break

        # Check robot routes
        for route in destroyed_state.robot_routes:
            if destroyed_state.remove_customer_from_route(customer, route):
                break

    return destroyed_state

def greedy_insert(state, random_state):
    """
    Insert unassigned customers using a greedy approach

    Args:
        state: Current solution state
        random_state: Random number generator

    Returns:
        TwoEchelonState: Modified solution with customers inserted
    """
    repaired_state = state.copy()

    # Process unassigned customers
    unassigned = repaired_state.unassigned.copy()

    for customer in unassigned:
        best_cost = float('inf')
        best_route = None
        best_position = -1
        is_robot_route = False

        # Check for robot-only customers
        if customer in customers_robot_only:
            # Check insertion in existing robot routes
            for i, route in enumerate(repaired_state.robot_routes):
                # Skip if adding this customer would exceed capacity
                if route.load + customer_demand[customer] > robot_params["capacity"]:
                    continue

                # Try all possible insertion positions
                for pos in range(1, len(route.nodes)):
                    # Create a temporary copy of the route
                    temp_route = route.copy()

                    # Insert customer at this position
                    temp_route.nodes.insert(pos, customer)
                    temp_route.update_metrics()

                    # Skip if energy constraint is violated
                    if temp_route.energy_consumption > robot_params["battery_capacity"]:
                        continue

                    # Calculate insertion cost
                    insertion_cost = temp_route.distance - route.distance

                    # Update best insertion if this is better
                    if insertion_cost < best_cost:
                        best_cost = insertion_cost
                        best_route = i
                        best_position = pos
                        is_robot_route = True

            # If no feasible insertion found, create a new robot route
            if best_route is None:
                # Choose a random station for start and end
                start_station = random_state.choice(charging_stations)
                end_station = random_state.choice(charging_stations)

                # Create new robot route
                new_route = Route('robot', [start_station, customer, end_station])
                new_route.load = customer_demand[customer]
                new_route.update_metrics()

                # Check if feasible
                if new_route.load <= robot_params["capacity"] and new_route.energy_consumption <= robot_params["battery_capacity"]:
                    repaired_state.robot_routes.append(new_route)
                    repaired_state.unassigned.remove(customer)

        else:  # Customer can be served by van or robot
            # Check insertion in van routes
            for i, route in enumerate(repaired_state.van_routes):
                # Skip if adding this customer would exceed capacity
                if route.load + customer_demand[customer] > van_params["capacity"]:
                    continue

                # Try all possible insertion positions
                for pos in range(1, len(route.nodes)):
                    # Create a temporary copy of the route
                    temp_route = route.copy()

                    # Insert customer at this position
                    temp_route.nodes.insert(pos, customer)
                    temp_route.update_metrics()

                    # Skip if energy constraint is violated
                    if temp_route.energy_consumption > van_params["battery_capacity"]:
                        continue

                    # Calculate insertion cost
                    insertion_cost = temp_route.distance - route.distance

                    # Update best insertion if this is better
                    if insertion_cost < best_cost:
                        best_cost = insertion_cost
                        best_route = i
                        best_position = pos
                        is_robot_route = False

            # Check insertion in robot routes
            for i, route in enumerate(repaired_state.robot_routes):
                # Skip if adding this customer would exceed capacity
                if route.load + customer_demand[customer] > robot_params["capacity"]:
                    continue

                # Try all possible insertion positions
                for pos in range(1, len(route.nodes)):
                    # Create a temporary copy of the route
                    temp_route = route.copy()

                    # Insert customer at this position
                    temp_route.nodes.insert(pos, customer)
                    temp_route.update_metrics()

                    # Skip if energy constraint is violated
                    if temp_route.energy_consumption > robot_params["battery_capacity"]:
                        continue

                    # Calculate insertion cost
                    insertion_cost = temp_route.distance - route.distance

                    # Update best insertion if this is better
                    if insertion_cost < best_cost:
                        best_cost = insertion_cost
                        best_route = i
                        best_position = pos
                        is_robot_route = True

        # Execute the best insertion if found
        if best_route is not None:
            if is_robot_route:
                repaired_state.add_customer_to_route(customer, repaired_state.robot_routes[best_route], best_position)
            else:
                repaired_state.add_customer_to_route(customer, repaired_state.van_routes[best_route], best_position)

    return repaired_state

def regret_insert(state, random_state):
    """
    Insert unassigned customers using a regret-based approach

    Args:
        state: Current solution state
        random_state: Random number generator

    Returns:
        TwoEchelonState: Modified solution with customers inserted
    """
    repaired_state = state.copy()

    while repaired_state.unassigned:
        best_regret = -float('inf')
        best_customer = None
        best_route = None
        best_position = -1
        is_robot_route = False

        # Compute regret values for each unassigned customer
        for customer in repaired_state.unassigned:
            # Store costs for all possible insertions
            insertion_costs = []
            insertion_details = []

            # Check if customer is robot-only or can be served by either
            if customer in customers_robot_only:
                # Check insertion in robot routes
                for i, route in enumerate(repaired_state.robot_routes):
                    # Skip if adding this customer would exceed capacity
                    if route.load + customer_demand[customer] > robot_params["capacity"]:
                        continue

                    # Try all possible insertion positions
                    for pos in range(1, len(route.nodes)):
                        # Create a temporary copy of the route
                        temp_route = route.copy()

                        # Insert customer at this position
                        temp_route.nodes.insert(pos, customer)
                        temp_route.update_metrics()

                        # Skip if energy constraint is violated
                        if temp_route.energy_consumption > robot_params["battery_capacity"]:
                            continue

                        # Calculate insertion cost
                        insertion_cost = temp_route.distance - route.distance

                        insertion_costs.append(insertion_cost)
                        insertion_details.append((insertion_cost, i, pos, True))  # True for robot route

            else:  # Customer can be served by van or robot
                # Check insertion in van routes
                for i, route in enumerate(repaired_state.van_routes):
                    # Skip if adding this customer would exceed capacity
                    if route.load + customer_demand[customer] > van_params["capacity"]:
                        continue

                    # Try all possible insertion positions
                    for pos in range(1, len(route.nodes)):
                        # Create a temporary copy of the route
                        temp_route = route.copy()

                        # Insert customer at this position
                        temp_route.nodes.insert(pos, customer)
                        temp_route.update_metrics()

                        # Skip if energy constraint is violated
                        if temp_route.energy_consumption > van_params["battery_capacity"]:
                            continue

                        # Calculate insertion cost
                        insertion_cost = temp_route.distance - route.distance

                        insertion_costs.append(insertion_cost)
                        insertion_details.append((insertion_cost, i, pos, False))  # False for van route

                # Check insertion in robot routes
                for i, route in enumerate(repaired_state.robot_routes):
                    # Skip if adding this customer would exceed capacity
                    if route.load + customer_demand[customer] > robot_params["capacity"]:
                        continue

                    # Try all possible insertion positions
                    for pos in range(1, len(route.nodes)):
                        # Create a temporary copy of the route
                        temp_route = route.copy()

                        # Insert customer at this position
                        temp_route.nodes.insert(pos, customer)
                        temp_route.update_metrics()

                        # Skip if energy constraint is violated
                        if temp_route.energy_consumption > robot_params["battery_capacity"]:
                            continue

                        # Calculate insertion cost
                        insertion_cost = temp_route.distance - route.distance

                        insertion_costs.append(insertion_cost)
                        insertion_details.append((insertion_cost, i, pos, True))  # True for robot route

            # Calculate regret value for this customer
            if len(insertion_costs) >= 2:
                # Sort insertion costs
                insertion_costs.sort()

                # Regret value is the difference between the best and second-best insertion
                regret_value = insertion_costs[1] - insertion_costs[0]

                # Find details of the best insertion
                best_detail = min(insertion_details, key=lambda x: x[0])

                if regret_value > best_regret:
                    best_regret = regret_value
                    best_customer = customer
                    best_route = best_detail[1]
                    best_position = best_detail[2]
                    is_robot_route = best_detail[3]

            elif len(insertion_costs) == 1:
                # Only one insertion possibility, use that as regret value
                best_detail = insertion_details[0]

                if best_detail[0] > best_regret:
                    best_regret = best_detail[0]
                    best_customer = customer
                    best_route = best_detail[1]
                    best_position = best_detail[2]
                    is_robot_route = best_detail[3]

        # If no feasible insertion found, break
        if best_customer is None:
            break

        # Execute the best insertion
        if is_robot_route:
            repaired_state.add_customer_to_route(best_customer, repaired_state.robot_routes[best_route], best_position)
        else:
            repaired_state.add_customer_to_route(best_customer, repaired_state.van_routes[best_route], best_position)

    return repaired_state

def run_alns():
    """
    Run the ALNS algorithm to solve the Two-Echelon Electric Vehicle Routing Problem

    Returns:
        TwoEchelonState: The best solution found
    """
    # Create initial solution
    initial_state = TwoEchelonState().create_initial_solution()

    # Initialize the ALNS object
    alns = ALNS()

    # Add destroy operators
    alns.add_destroy_operator(random_remove)
    alns.add_destroy_operator(worst_remove)

    # Add repair operators
    alns.add_repair_operator(greedy_insert)
    alns.add_repair_operator(regret_insert)

    # Set accept and stop criteria
    accept = SimulatedAnnealing(
        start_temperature=100,
        end_temperature=1,
        step=0.99,  # Must be less than 1 for exponential cooling
        method="exponential"
    )

    select = RouletteWheel([25, 5, 1, 0], 0.8, 1, 1)

    stop = MaxIterations(100)

    # Run the ALNS algorithm
    result = alns.iterate(initial_state, select, accept, stop)

    # Return the best solution
    return result.best_state

# Execute the algorithm
if __name__ == "__main__":
    solution = run_alns()

    print("\n===== Two-Echelon Electric Vehicle Routing Solution =====")
    print(f"Objective Value: {solution.objective()}")
    print("\nVan Routes:")
    for i, route in enumerate(solution.van_routes):
        print(f"  Route {i+1}: {route.nodes} (Distance: {route.distance:.2f}, Load: {route.load})")

    print("\nRobot Routes:")
    for i, route in enumerate(solution.robot_routes):
        print(f"  Route {i+1}: {route.nodes} (Distance: {route.distance:.2f}, Load: {route.load})")

    print(f"\nUnassigned Customers: {solution.unassigned}")


===== Two-Echelon Electric Vehicle Routing Solution =====
Objective Value: inf

Van Routes:
  Route 1: [0, 1, 2, 3, 4, 5, 0] (Distance: 300.00, Load: 0)

Robot Routes:
  Route 1: [1, 6, 1] (Distance: 100.00, Load: 15)
  Route 2: [2, 8, 4] (Distance: 110.00, Load: 10)
  Route 3: [3, 7, 1] (Distance: 100.00, Load: 20)
  Route 4: [4, 9, 4] (Distance: 60.00, Load: 25)

Unassigned Customers: []
